<h1>Index<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fixing" data-toc-modified-id="Fixing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fixing</a></span></li></ul></div>

In [2]:
import pandas as pd
import numpy as np
import pdb
import requests
import time
import random
import os
import glob
import sys
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select
from tqdm import tqdm_notebook, tqdm
from joblib import Parallel, delayed

In [4]:
SAVING_PATH = "~/Documents/datasets/cartola"

In [5]:
TEAM_ABBREV_TO_TEAM_KEY = {"CAP": "athletico-pr",
                           "ACG": "atletico-go",
                           "ATL": "atletico-mg",
                           "BAH": "bahia",
                           "BGT": "bragantino",
                           "BOT": "botafogo",
                           "CEA": "ceara",
                           "COR": "corinthians",
                           "CFC": "coritiba",
                           "FLA": "flamengo",
                           "FLU": "fluminense",
                           "FOR": "fortaleza",
                           "GOI": "goias",
                           "GRE": "gremio",
                           "INT": "internacional",
                           "PAL": "palmeiras",
                           "SAN": "santos",
                           "SAO": "sao-paulo",
                           "SPO": "sport",
                           "VAS": "vasco"}

TEAM_LONG_TO_TEAM_KEY = {"Athlético-PR": "athletico-pr",
                         "Atlético-GO": "atletico-go",
                         "Atlético-MG": "atletico-mg",
                         "Bahia": "bahia",
                         "Bragantino": "bragantino",
                         "Botafogo": "botafogo",
                         "Ceará": "ceara",
                         "Corinthians": "corinthians",
                         "Coritiba": "coritiba",
                         "Flamengo": "flamengo",
                         "Fluminense": "fluminense",
                         "Fortaleza": "fortaleza",
                         "Goiás": "goias",
                         "Grêmio": "gremio",
                         "Internacional": "internacional",
                         "Palmeiras": "palmeiras",
                         "Santos": "santos",
                         "São Paulo": "sao-paulo",
                         "Sport": "sport",
                         "Vasco": "vasco"}



In [56]:
def go_to_page_number(driver, page_number):
    try:
        driver.find_element_by_link_text(str(page_number)).click()
        time.sleep(10)
        return driver
    except:
        return driver
    

def get_page_numbers(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    tr = soup.find("tr", {"class": "tbpaging"})
    tr = tr.find("tr")
    tds = tr.find_all("td")
    page_numbers = [td.text for td in tds]
    
    return page_numbers

    
def get_table_from_page(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')

#    table = soup.findAll("table", {"class": "gridView"})
    table = soup.find("table", {"id": "ctl00_cphMainContent_gvList"})
    df = pd.read_html(str(table))[0]
    df_n_rows = df.shape[0]
    df = df.drop([df_n_rows-2, df_n_rows-1])
    imgs = table.find_all("img")
    clubs = [img.__dict__["attrs"]["alt"] for img in imgs]
    df["Clube"] = clubs
    return df

def input_element_id_with(driver, element_id, value):
    dropdown = Select(driver.find_element_by_id(element_id)) 
    dropdown.select_by_visible_text(value)
    
    return driver
    
def input_search_and_click(search, driver):
    for key, value in search.items():
        input_element_id_with(driver, key, value) 
    
    ### Click the filter button    
    driver.find_element_by_id("ctl00_cphMainContent_btnFiltrar").click()
    time.sleep(10)
    
    return driver

def get_all_tables_from_driver(driver):
    #drive = input_search_and_click(search, driver)
    
    page_numbers = get_page_numbers(driver)
            
    infos = []
    
    for page_number in page_numbers:
        go_to_page_number(driver, page_number)
        df = get_table_from_page(driver)
        infos.append(df)

        
    return pd.concat(infos)


def go_to_specific_tab(driver, tab_name):
    try:
        driver.find_element_by_link_text(tab_name).click()
        time.sleep(10)
        return driver
    except:
        return driver

def get_all_div_id_table_rows(driver, div_id):
    soup = BeautifulSoup(driver.page_source, 'lxml')    
    table = soup.find("div", {"id": div_id})
        
    return table.find_all("a", recursive=True)

def from_game_table_row_to_dict(n_row, row):
    row_data = {}
    row_data["mandante"] = row.__dict__["attrs"]["title"].split(" x ")[0]
    row_data["visitante"] = row.__dict__["attrs"]["title"].split(" x ")[1].split(" - ")[0]
    row_data["placar_mandante"] = row.find("span", {"id": "ctl00_cphMainContent_rptListPartidas_ctl{}_lblPlacarMandante".format(str(n_row).zfill(2))}).text
    row_data["placar_visitante"] = row.find("span", {"id": "ctl00_cphMainContent_rptListPartidas_ctl{}_lblPlacarVisitante".format(str(n_row).zfill(2))}).text
    row_data["data"] = row.find_all("strong")[0].text.replace("\n", "").strip()
    row_data["local"] = row.find_all("span", {"class": "text-muted"})[0].text.replace("\n", "").strip()
    
    return row_data

def extract_games_result(driver):
    go_to_specific_tab(driver, "Partidas")
    rows = get_all_div_id_table_rows(driver, "tab2default")
    games = []
    for n_row, row in enumerate(rows):    
        row_data = from_game_table_row_to_dict(n_row, row)
        games.append(row_data)

    games = pd.DataFrame.from_dict(games)
    return games

def from_player_table_row_to_dict(n_row, row):
    row_data = {}
    row_data["colocação"] = n_row + 1
    row_data["jogador"] = row.find("span", {"id": "ctl00_cphMainContent_rptListMaisEscalados_ctl{}_lblNome".format(str(n_row).zfill(2))}).text
    row_data["escalações"] = row.find("span", {"id": "ctl00_cphMainContent_rptListMaisEscalados_ctl{}_lblQtdEscalacoes".format(str(n_row).zfill(2))}).text.replace("escalações", "").replace(".", "")

    
    return row_data

def extract_most_chosen_players(driver):
    go_to_specific_tab(driver, "Mais escalados")
    rows = get_all_div_id_table_rows(driver, "tab3default") 
    players = []
    for n_row, row in enumerate(rows):    
        row_data = from_player_table_row_to_dict(n_row, row)
        players.append(row_data)
    
    players = pd.DataFrame.from_dict(players)
    players["escalações"] = players["escalações"].astype(int)
    return players
    

def get_data_from_championship_round(year, championship_round, search, driver=None, close_driver=False):
    page_prefix = "https://www.cartolafcbrasil.com.br/scouts/cartola-fc-{}/rodada-{}"
    page = page_prefix.format(year, championship_round)
    
    if not driver:
        options = Options()
        options.add_argument("--headless")
        driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
        driver.implicitly_wait(20)
        
    driver.get(page)    
    driver = input_search_and_click(search, driver)
    data = get_all_tables_from_driver(driver)
    
    if close_driver: driver.close()
        
    return data

def get_all_year_data(year, max_round, search):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
    driver.implicitly_wait(20)
    
    all_players = []
    all_games = []
    all_uses = []
    for ch_round in range(1, max_round+1):
        players = get_data_from_championship_round(year,
                                         ch_round,
                                         search,
                                         driver=driver)       
        games = extract_games_result(driver)
        uses = extract_most_chosen_players(driver)
        players["rodada"] = ch_round
        games["rodada"] = ch_round
        uses["rodada"] = ch_round
        
        all_players.append(players)
        all_games.append(games)
        all_uses.append(uses)
        
    all_players = pd.concat(all_players)
    all_games = pd.concat(all_games)
    all_uses = pd.concat(all_uses)
    
    all_players["ano"] = year
    all_games["ano"] = year
    all_uses["ano"] = year
    
    driver.close()
    
    return all_players, all_games, all_uses

In [21]:
search = {"ctl00_cphMainContent_drpStatus": "[TODOS]"}

In [52]:
page_prefix = "https://www.cartolafcbrasil.com.br/scouts/cartola-fc-{}/rodada-{}"
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
driver.implicitly_wait(5)

year = 2020
championship_round = 1
page = page_prefix.format(year, championship_round)
driver.get(page)

In [53]:
games = extract_games_result(driver)
players = extract_most_chosen_players(driver)
driver.close()

In [54]:
players

,colocação,jogador,escalações
0,1,Gabriel (FLA),3327349
1,2,Pedro Geromel (GRE),3127228
2,3,Everton Ribeiro (FLA),2840557
3,4,Vanderlei (GRE),1969166
4,5,Marinho (SAN),1850604
5,6,Thiago Galhardo (INT),1422202
6,7,Soteldo (SAN),1308558
7,8,Víctor Cuesta (INT),1304007
8,9,Diego Souza (GRE),1273009
9,10,Alisson (GRE),1247015


In [55]:
games

,mandante,visitante,placar_mandante,placar_visitante,data,local
0,Fortaleza,Athlético-PR,0,2,"SÁB, 08/08 19:00",Castelão (CE)
1,Coritiba,Internacional,0,1,"SÁB, 08/08 19:30",Couto Pereira
2,Sport,Ceará,3,2,"SÁB, 08/08 21:00",Ilha do Retiro
3,Botafogo,Bahia,,,"DOM, 09/08 11:00",JOGO ADIADO* (Partida não válida para a rodada)
4,Flamengo,Atlético-MG,0,1,"DOM, 09/08 16:00",Maracanã
5,Santos,Bragantino,1,1,"DOM, 09/08 16:00",Vila Belmiro
6,Corinthians,Atlético-GO,,,"DOM, 09/08 16:00",JOGO ADIADO* (Partida não válida para a rodada)
7,Goiás,São Paulo,,,"DOM, 09/08 16:00",JOGO ADIADO* (Partida não válida para a rodada)
8,Grêmio,Fluminense,1,0,"DOM, 09/08 19:00",Arena do Grêmio
9,Palmeiras,Vasco,,,"DOM, 09/08 19:45",JOGO ADIADO* (Partida não válida para a rodada)


In [24]:
search

{'ctl00_cphMainContent_drpStatus': '[TODOS]'}

In [46]:
search = {"ctl00_cphMainContent_drpStatus": "Provável"}
data = get_data_from_championship_round(2020, 1, search, driver=None,
                                       close_driver=True)

In [47]:
data.head()

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,DS,G,A,...,DD,DP,GC,CV,CA,PP,GS,FC,I,PI
0,Elton (ATA),Sport,1421,1,1560,1560.0,1021.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,4.0
1,Saravia (LAT),Internacional,1319,1,1220,1220.0,719.0,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,8.0
2,Marinho (ATA),Santos,1210,1,1170,1170.0,710.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,5.0
3,Igor Rabello (ZAG),Atlético-MG,1341,1,1050,1050.0,541.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
4,Léo Cittadini (MEI),Athlético-PR,1290,1,980,980.0,490.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.shape

In [36]:
data = get_all_year_data(2020, 10, search)

In [37]:
data.shape

(7134, 26)

In [38]:
data

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,DS,G,A,...,DP,GC,CV,CA,PP,GS,FC,I,PI,rodada
0,Elton (ATA),NaN,1421,1,1560,1560,1021,NaN,2,NaN,...,NaN,NaN,NaN,1,NaN,NaN,1,1,4,1
1,Saravia (LAT),NaN,1319,1,1220,1220,719,8,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,3,NaN,8,1
2,Marinho (ATA),NaN,1210,1,1170,1170,710,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,5,1
3,Vitinho (ATA),NaN,1029,1,1120,1120,729,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1
4,Igor Rabello (ZAG),NaN,1341,1,1050,1050,541,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,Luan Polli (GOL),NaN,271,4,-258,-520,-071,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,5,NaN,NaN,63,10
12,Kawan (MEI),NaN,072,1,-260,000,000,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,1,NaN,1,10
13,Jailson (GOL),NaN,306,1,-310,000,000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,11,10
14,Marcelo Rangel (GOL),NaN,077,2,-345,-200,-009,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,3,NaN,NaN,29,10


In [61]:
players, games, uses = get_all_year_data(2020, 10, search)

In [62]:
players.head()

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,DS,G,A,...,DP,GC,CV,CA,PP,GS,FC,I,PI,rodada
0,Elton (ATA),Sport,1421,1,1560.0,1560.0,1021.0,NaN,2.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,4.0,1
1,Saravia (LAT),Internacional,1319,1,1220.0,1220.0,719.0,8.0,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,8.0,1
2,Marinho (ATA),Santos,1210,1,1170.0,1170.0,710.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,5.0,1
3,Igor Rabello (ZAG),Atlético-MG,1341,1,1050.0,1050.0,541.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1
4,Léo Cittadini (MEI),Athlético-PR,1290,1,980.0,980.0,490.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [63]:
games.head()

,mandante,visitante,placar_mandante,placar_visitante,data,local,rodada
0,Fortaleza,Athlético-PR,0,2,"SÁB, 08/08 19:00",Castelão (CE),1
1,Coritiba,Internacional,0,1,"SÁB, 08/08 19:30",Couto Pereira,1
2,Sport,Ceará,3,2,"SÁB, 08/08 21:00",Ilha do Retiro,1
3,Botafogo,Bahia,,,"DOM, 09/08 11:00",JOGO ADIADO* (Partida não válida para a rodada),1
4,Flamengo,Atlético-MG,0,1,"DOM, 09/08 16:00",Maracanã,1


In [64]:
uses.head()

,colocação,jogador,escalações,rodada
0,1,Gabriel (FLA),3327349,1
1,2,Pedro Geromel (GRE),3127228,1
2,3,Everton Ribeiro (FLA),2840557,1
3,4,Vanderlei (GRE),1969166,1
4,5,Marinho (SAN),1850604,1


In [67]:
uses["ano"] = 2020
games["ano"] = 2020
players["ano"] = 2020

In [68]:
players_.to_csv("jogadores_2020.csv", index=False)
games_.to_csv("jogos_2020.csv", index=False)
uses_.to_csv("escalacoes_2020.csv", index=False)

In [69]:
players.columns

Index(['Nome', 'Clube', 'Preço', 'J', 'Média', 'Últ. Pont.', 'Variação', 'DS',
       'G', 'A', 'SG', 'FS', 'FF', 'FD', 'FT', 'DD', 'DP', 'GC', 'CV', 'CA',
       'PP', 'GS', 'FC', 'I', 'PI', 'rodada', 'ano'],
      dtype='object')

In [70]:
players, games, uses = get_all_year_data(2019, 38, search)

In [72]:
uses["ano"] = 2019
games["ano"] = 2019
players["ano"] = 2019

In [77]:
players.to_csv("jogadores_2019.csv", index=False)
games.to_csv("jogos_2019.csv", index=False)
uses.to_csv("escalacoes_2019.csv", index=False)

In [74]:
players_ = pd.read_csv("jogadores.csv", index_col=False)
games_ = pd.read_csv("jogos.csv", index_col=False)
uses_ = pd.read_csv("escalacoes.csv", index_col=False)

In [76]:
final_players = pd.concat([players, players_])
final_games = pd.concat([games, games_])
final_uses = pd.concat([uses, uses_])

In [78]:
final_players.to_csv("jogadores.csv", index=False)
final_games.to_csv("jogos.csv", index=False)
final_uses.to_csv("escalacoes.csv", index=False)

In [79]:
final_players

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,RB,G,A,...,CA,PP,GS,FC,I,PE,rodada,ano,DS,PI
0,Ricardo Bueno (ATA),Ceará,1852,1,2270.0,2270.0,1452.0,1.0,2.0,1.0,...,NaN,NaN,NaN,2.0,1.0,1.0,1,2019,NaN,NaN
1,Bruno Henrique (ATA),Flamengo,2151,1,2100.0,2100.0,1151.0,NaN,2.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1,2019,NaN,NaN
2,Nikão (MEI),Athlético-PR,1929,1,1880.0,1880.0,1029.0,5.0,1.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1,2019,NaN,NaN
3,Tiepo (GOL),Chapecoense,1318,1,1700.0,1700.0,1118.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2019,NaN,NaN
4,Vanderlei (GOL),Santos,2269,1,1570.0,1570.0,569.0,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,1.0,1,2019,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,Weverton (GOL),Palmeiras,709,8,-9.0,-400.0,-170.0,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,10,2020,NaN,47.0
2139,Matheus Henrique (MEI),Grêmio,532,6,-32.0,0.0,0.0,NaN,NaN,NaN,...,2.0,NaN,NaN,9.0,NaN,NaN,10,2020,9.0,37.0
2140,Pedro Henrique (ZAG),Internacional,77,1,-100.0,-100.0,-23.0,NaN,NaN,NaN,...,1.0,NaN,NaN,2.0,NaN,NaN,10,2020,1.0,7.0
2141,Weverson (LAT),Bragantino,76,6,-103.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,1.0,NaN,10,2020,1.0,12.0


## Fixing

In [4]:
final_players = pd.read_csv("jogadores.csv", index_col=False)
final_games = pd.read_csv("jogos.csv", index_col=False)
final_uses = pd.read_csv("escalacoes.csv", index_col=False)

In [6]:
final_players[["Preço", "Média", "Últ. Pont.", "Variação"]] = final_players[["Preço", "Média", "Últ. Pont.", "Variação"]] / 100.0

In [8]:
final_players.to_csv("jogadores.csv", index=False)